In [45]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


df = pd.read_excel("zbior_punktow.xlsx", 'Arkusz2')
dct = dict()
pkt_nazwa = list(df['U'])
pkt_x = list(df['x'])
pkt_y = list(df['y'])
for i in range(len(pkt_x)):
    dct[pkt_nazwa[i]] = (pkt_x[i], pkt_y[i])

df2 = pd.read_excel("zbior_punktow.xlsx", 'Arkusz3')

A0_idx = df2.A0.to_list()
A1_idx = df2.A1.to_list()
A2_idx = df2.A2.to_list()
A3_idx = df2.A3.to_list()

A0 = []
A1 = []
A2 = []
A3 = []
for i in range(len(A0_idx)):
    A0.append(dct[A0_idx[i]])
    A1.append(dct[A1_idx[i]])
    A2.append(dct[A2_idx[i]])
    A3.append(dct[A3_idx[i]])


B0 = []


A1xmin = min([point[0] for point in A1])
A2xmax = max([point[0] for point in A2])

for point_name in dct.keys():
    if A1xmin < dct[point_name][0] < A2xmax:
        if (dct[point_name] not in A1) and (dct[point_name] not in A2) and (dct[point_name] not in A0) and (dct[point_name] not in A3):
            B0.append(dct[point_name])

In [46]:
def oblicz_d(A1_point,A2_point):
    x_A1, y_A1 = A1_point
    x_A2, y_A2 = A2_point
    d1 = (x_A1 - x_A2)/2
    d2 = (y_A1 - y_A2)/2
    return min(d1,d2)

def stworz_krzywa(A1_point,A2_point):
    d = oblicz_d(A1_point,A2_point)
    x_A1, y_A1 = A1_point
    x_A2, y_A2 = A2_point
    f1 = (x_A1-d,y_A1-d)
    f2 = (x_A2+d,y_A2+d)
    return (A1_point,f1,f2,A2_point)

def oblicz_odleglosc(u,A1_point,A2_point):
    A1_point, f1, f2, A2_point = stworz_krzywa(A1_point,A2_point)
    lista = [A1_point, f1, f2, A2_point]
    x_u, y_u = u
    odle = []
    for i in range(3):
        if lista[i][0]-lista[i+1][0] == 0:
            A = 0
            B = -1
            C = 0
        else: 
            B = -1
            A = (lista[i][1]-lista[i+1][1])/(lista[i][0]-lista[i+1][0])
            C = lista[i][1] - lista[i][0]*(lista[i][1]-lista[i+1][1])/(lista[i][0]-lista[i+1][0])
        odle.append((abs(A*x_u + B*y_u + C))/(A**2+B**2)**(1/2))
    return min(odle),odle.index(min(odle)), f1, f2

# print(oblicz_d(dct["N22"],dct['N12']))
# print(oblicz_odleglosc(dct['N22'],dct['N12'],dct['N5']))

def odleglosc(point_1,point_2):
    x_1, y_1 = point_1
    x_2, y_2 = point_2
    return ((x_2-x_1)**2 + (y_2 - y_1)**2)**(1/2)

def oblicz_wspolczynnik_skoringowy(u,A1_point,A2_point):
    odle, idx, f1, f2 = oblicz_odleglosc(u,A1_point,A2_point)
    odleglosci = [odleglosc(A1_point,f1),odleglosc(f1,f2),odleglosc(f2,A2_point)]
    suma = odleglosci[0] + odleglosci[1] + odleglosci[2]
    waga = odleglosci[idx]/suma
    wsp_skoringowy = odle * waga
    return wsp_skoringowy

ranking = []

for point in B0:
    suma = 0
    for A1_point in A1:
        for A2_point in A2:
            suma += oblicz_wspolczynnik_skoringowy(point,A1_point,A2_point)
    point = list(dct.keys())[list(dct.values()).index((point[0], point[1]))]
    ranking.append((point,suma))

ranking = sorted(ranking, key = lambda tup: tup[1], reverse=False)

print(ranking)

[('N26', 12.971473656276304), ('N23', 13.960569169038278), ('N2', 14.430563575273904), ('N8', 15.591988015855483), ('N24', 17.680804243748234), ('N20', 19.28305190314624), ('N6', 20.479519921551123), ('N7', 21.957242009969566), ('N1', 22.112310872498604), ('N17', 23.199644856791792), ('N27', 23.416565406129514), ('N22', 23.690970999616816), ('N3', 24.164995063139727), ('N14', 27.254944166073162), ('N21', 28.406735333990213)]
